<a href="https://colab.research.google.com/github/Explorer9/INFO5731_FALL2020/blob/master/lda2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# Write your code here
import pandas as pd
import requests
from bs4 import BeautifulSoup
def insert_into_array(values,ref_array):
  for i in values:
    ref_array.append(i.text)
URL=["https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7djfquydczye7wthxnztqltmwcbhzfmxvlnomwklyczuf43o6ssypazfjmjddv4k4rk5izrbr2k7kbghholct5fuigy","https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7dbfqi3tizqg66uhhnzrrdt4ucbyy4hhzo5ziwr26fbyhvrl4ty4oqyvtprodfr5dtt3nlozdkpd6sssetvth324k3g7","https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7dbqsr3ximsatx5dz2dezxzyyrr72intd6pnaljzoplumbhlqx7ok6yfifxz3a76iui","https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7dbqsr3ximsatx5dz2dezxzyyrr73mptr4hfapjzpeaaaj5kx7i75r5y6odr55p7f2q"]
text=[]
rating=[]
review_title=[]
for i in URL:
  page=requests.get(i,headers={'User-Agent':'Chrome/85.0.4183.121'})
  soup = BeautifulSoup(page.content, 'html.parser')
  insert_into_array(soup.find_all('div', class_='text show-more__control'),text)
  insert_into_array(soup.find_all('a',class_='title'),review_title)
  insert_into_array(soup.find_all(name='span',class_='rating-other-user-rating'),rating)
imdb_df=pd.DataFrame({'comment':text,'review_title':review_title})
imdb_df.to_csv("imbd")
print(imdb_df[:6])

                                             comment                                       review_title
0  I can't understand what's so path breaking, in...                                 A wannabe Devdas\n
1  I have personally known people like Arjun Redd...   Arjun Reddy is a person who sees world in bla...
2  Best acting , mind blowing background scores d...    One of the realistic movie in Telugu industry\n
3  If "Rashmika Mandanna" was Preethi\nIt would b...                                               IF\n
4  Arjun Reddy is a story in purest. Its fresh, l...                   A milestone in romantic genre.\n
5  The movie displays plain and uncensored drug a...                                          Unique.\n


In [2]:
from helper import *

In [3]:
! pip install pandas nltk gensim pyldavis

     |████████████████████████████████| 1.6MB 2.7MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=db5d427daea3bb6a39bc23d799663a60935237678b161db578cddd08fcbce658
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
text = imdb_df.applymap(cleaning)['comment']
text_list = [i.split() for i in text]
len(text_list)

80

In [8]:
from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='running.log',filemode='w')

In [10]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(text_list)
dictionary.save('dictionary.dict')
print(dictionary)

Dictionary(1543 unique tokens: ['afraid', 'again', 'among', 'anger', 'anytime']...)


In [13]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

print(len(doc_term_matrix))
print(doc_term_matrix[70])

80
[(3, 1), (6, 11), (7, 1), (16, 1), (18, 1), (26, 1), (32, 7), (35, 1), (38, 1), (40, 2), (41, 1), (46, 1), (50, 2), (52, 4), (56, 1), (58, 1), (60, 1), (61, 2), (68, 2), (69, 6), (73, 1), (74, 1), (75, 1), (77, 2), (81, 1), (82, 2), (83, 3), (88, 4), (90, 2), (115, 1), (117, 1), (118, 3), (119, 1), (120, 1), (125, 6), (128, 1), (130, 1), (132, 1), (138, 1), (144, 1), (149, 1), (155, 1), (159, 1), (171, 6), (173, 1), (174, 1), (186, 1), (188, 2), (198, 1), (222, 1), (223, 1), (237, 1), (239, 1), (250, 1), (273, 4), (274, 1), (280, 1), (283, 1), (284, 1), (293, 1), (302, 1), (303, 1), (311, 1), (340, 1), (344, 1), (354, 1), (373, 1), (386, 1), (392, 1), (409, 1), (414, 1), (418, 1), (430, 1), (438, 1), (470, 1), (471, 3), (479, 2), (481, 1), (492, 1), (524, 1), (536, 1), (578, 1), (580, 1), (584, 1), (594, 1), (612, 1), (623, 3), (647, 2), (671, 2), (680, 1), (684, 2), (708, 2), (710, 1), (714, 2), (725, 1), (734, 2), (739, 1), (748, 1), (751, 2), (753, 2), (766, 1), (787, 1), (807, 1

In [15]:
start = time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)
print('used: {:.2f}s'.format(time()-start))

used: 2.57s


In [16]:
print(ldamodel.print_topics(num_topics=2, num_words=4))

[(2, '0.022*"good" + 0.022*"movie" + 0.006*"direction" + 0.006*"here"'), (5, '0.021*"movie" + 0.015*"love" + 0.013*"story" + 0.011*"like"')]


In [18]:
for i in ldamodel.print_topics(): 
    for j in i:
      print(j)

0
0.037*"movie" + 0.012*"acting" + 0.010*"better" + 0.010*"vijay" + 0.010*"performance" + 0.010*"character" + 0.010*"good" + 0.010*"arjun" + 0.008*"director" + 0.008*"every"
1
0.029*"movie" + 0.015*"reddy" + 0.015*"time" + 0.012*"vijay" + 0.012*"like" + 0.012*"arjun" + 0.009*"loved" + 0.009*"way" + 0.009*"good" + 0.009*"film"
2
0.022*"good" + 0.022*"movie" + 0.006*"direction" + 0.006*"here" + 0.006*"job" + 0.006*"hour" + 0.006*"much" + 0.006*"also" + 0.006*"overall" + 0.006*"say"
3
0.022*"movie" + 0.019*"arjun" + 0.016*"vijay" + 0.013*"reddy" + 0.013*"character" + 0.011*"love" + 0.011*"film" + 0.009*"life" + 0.009*"deverakonda" + 0.008*"lead"
4
0.045*"movie" + 0.013*"arjun" + 0.011*"story" + 0.011*"film" + 0.011*"telugu" + 0.011*"love" + 0.008*"also" + 0.008*"way" + 0.007*"one" + 0.007*"language"
5
0.021*"movie" + 0.015*"love" + 0.013*"story" + 0.011*"like" + 0.011*"reddy" + 0.011*"arjun" + 0.008*"ending" + 0.008*"character" + 0.007*"end" + 0.007*"everything"
6
0.024*"arjun" + 0.022*"m

In [19]:
ldamodel.save('topic.model')

In [20]:
from gensim.models import LdaModel
loading = LdaModel.load('topic.model')

In [21]:
print(loading.print_topics(num_topics=2, num_words=4))

[(3, '0.022*"movie" + 0.019*"arjun" + 0.016*"vijay" + 0.013*"reddy"'), (6, '0.024*"arjun" + 0.022*"movie" + 0.014*"reddy" + 0.009*"telugu"')]


In [22]:
def pre_new(doc):
    one = cleaning(doc).split()
    two = dictionary.doc2bow(one)
    return two

In [23]:
pre_new('new article that to be classified by trained model!')

[(58, 1), (593, 1)]

In [24]:
belong = loading[(pre_new('new article that to be classified by trained model!'))]
belong

[(0, 0.033333335),
 (1, 0.033333335),
 (2, 0.033333335),
 (3, 0.033335064),
 (4, 0.6999851),
 (5, 0.033333335),
 (6, 0.033339743),
 (7, 0.033333335),
 (8, 0.033336114),
 (9, 0.03333726)]

In [25]:
new = pd.DataFrame(belong,columns=['id','prob']).sort_values('prob',ascending=False)
new['topic'] = new['id'].apply(loading.print_topic)
new

,id,prob,topic
4,4,0.699985,"0.045*""movie"" + 0.013*""arjun"" + 0.011*""story"" ..."
6,6,0.033340,"0.024*""arjun"" + 0.022*""movie"" + 0.014*""reddy"" ..."
9,9,0.033337,"0.022*""film"" + 0.016*""reddy"" + 0.015*""love"" + ..."
8,8,0.033336,"0.052*""movie"" + 0.016*""reddy"" + 0.016*""arjun"" ..."
3,3,0.033335,"0.022*""movie"" + 0.019*""arjun"" + 0.016*""vijay"" ..."
0,0,0.033333,"0.037*""movie"" + 0.012*""acting"" + 0.010*""better..."
1,1,0.033333,"0.029*""movie"" + 0.015*""reddy"" + 0.015*""time"" +..."
2,2,0.033333,"0.022*""good"" + 0.022*""movie"" + 0.006*""directio..."
5,5,0.033333,"0.021*""movie"" + 0.015*""love"" + 0.013*""story"" +..."
7,7,0.033333,"0.016*""movie"" + 0.011*""actor"" + 0.011*""also"" +..."


In [27]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

In [28]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is

In [29]:
data = pyLDAvis.gensim.prepare(lda, c, d)
data

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.051549 -0.102980       1        1  22.610740
6      0.143196  0.064207       2        1  14.898366
4     -0.078287  0.119452       3        1  14.838838
8     -0.055029 -0.027051       4        1  12.537992
5     -0.005513 -0.003014       5        1  12.219759
9     -0.074065  0.052857       6        1   8.918884
0      0.023125 -0.069777       7        1   6.704222
1      0.032863 -0.016695       8        1   4.959057
7      0.027967 -0.005859       9        1   1.273760
2      0.037293 -0.011139      10        1   1.038381, topic_info=            Term        Freq       Total Category  logprob  loglift
155        movie  112.000000  112.000000  Default  30.0000  30.0000
127         good   19.000000   19.000000  Default  29.0000  29.0000
120         also   12.000000   12.000000  Default  28.0000  28.0000
83          time   10.000000   10.000000  Default  27.0000  27.0000
273  deverakonda    9.000000    9.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
155        movie    0.868526  112.004423  Topic10  -3.8217  -0.2920
88         vijay    0.233018   29.540952  Topic10  -5.1374  -0.2749
6          arjun    0.021184   59.389308  Topic10  -7.5353  -3.3711
69         reddy    0.021184   45.521030  Topic10  -7.5353  -3.1052
32          film    0.021184   38.117395  Topic10  -7.5353  -2.9277

[617 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
397       2  0.399382     2
397       8  0.399382     2
941       6  0.498964  2017
399       2  0.290756     3
399       3  0.290756     3
...     ...       ...   ...
734       1  0.768280  year
734       5  0.256093  year
355       4  0.727005    👍👍
1202      8  0.854008     💯
1170      4  0.727005    😍😍

[956 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 5, 9, 6, 10, 1, 2, 8, 3])

In [1]:
!pip install lda2vec

  Created wheel for lda2vec: filename=lda2vec-0.16.10-cp36-none-any.whl size=14432 sha256=90064116443bf34ac4b743965968171fe856e197442e01e2f26218e0dd5c244e
  Stored in directory: /root/.cache/pip/wheels/73/46/3c/d6d69eaa65fa03389b2d5b2e96bad4d8d09c670bdbab289e61
Successfully built lda2vec


In [8]:
load_embeds=True
import pandas as pd
from lda2vec.nlppipe import Preprocessor
P = Preprocessor(imdb_df, "comment", max_features=30000, maxlen=10000, min_count=30)
P.preprocess()
if load_embeds:
    # Load embedding matrix from file path - change path to where you saved them
    embedding_matrix = P.load_glove("/content/drive/My Drive/glove.6B.300d.txt")
else:
    embedding_matrix = None

80it [00:00, 822.32it/s]
80it [00:00, 1582.37it/s]


---------- Tokenizing Texts ----------
Removing 1629 low frequency tokens out of 1666 total tokens

---------- Getting Skipgrams ----------



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install -U git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis